# A continuación se describen los principales metodos de depuración y control de archivos a emplear en cualquier notebook

Pandas Cheats Sheet
link: https://www.dataquest.io/blog/pandas-cheat-sheet/

## 1) Procesamientos con expresiones regulares

In [1]:
import re #libreria

**Tratamiento de archivos**

In [18]:
patttern_to_delete = r'(\d{1,}\w|_)|(centrodese[A-Z][0-9]|centrodese)' # Ejemplo, si quiero quitar digitos seguidos de "_", y palabras especificas
texts = '1_144a_archivo_centrodeseT6.xlsx'
re.sub(patttern_to_delete,'',texts)

'archivo.xlsx'

**Definición de Patrones Especificos que se desea buscar**

In [11]:
pattern_for_it = re.compile(r'PA[0-9]{4,5}|IT[0-9]{4,5}|pa[0-9]{4,5}|it[0-9]{4,5}|P[0-9]{3,4}|p[0-9]{3,4}',re.X) #uso el compilador Verbose, para usar el /b "bounday de palabras"
pattern_for_field = re.compile(r'\b(?!CENTRAL|PERSON|FALSO|\n)[A-Z]{4,}\b',re.X)
pattern_for_country = re.compile(r'[A-Z]{4,}-MX|[A-Z]{4,}-AR|[A-Z]{4,}-NI|[A-Z]{4,}-CO|[A-Z]{4,}-COL|[A-Z]{4,}-NI|[A-Z]{4,}-BR|[A-Z]{4,}-VZ|[A-Z]{4,}-PA|[A-Z]{4,}-UY',re.X)
#el primer pattern busca, los PAy que acontinuación tengan cierta cantidad de caracteres, luego, separado por | los IT, con la misma estructura, 
#el segundo pattern, busco las palabras que empiecen en mayuscuca, a partir de 4 caracteres en adelante y excluyo ciertas palabras
#en el pattern de pais, solo busco los paises de forma literal

**Funciones helpers**

*Funcion para encontrar los patternes definidos en una columna de un dataframe e incorporación al row, concatenado*

In [ ]:
def search_patterns_in_columns(row, infotype, pattern):
    a = re.findall(pattern,str(row))
    s = ',' #string para hacer el join
    data = s.join(a)
    if infotype != "" or infotype != None:
        all_data = s.join([data,infotype])
        return all_data
    else:
        return data

*convertir un valor encontrado en un texto, segun diccionario, y devolver otro y completarlo en la columna requerida*

In [1]:
#defino un diccionario cualquiera, con clave y valor
countries_dict = {'Brasil': 'BR','Venezuela': 'VZ', 'México': 'MX', 'Colombia': 'CO', 'Filipinas': 'FI', 'Costa Rica':'CR', 'Guatemala': 'GM', 'Nicaragua': 'NI', 'Panamá' : 'PA','Argentina': 'AR'}

In [2]:
def convert_text_to_iso_country(row, column):
    s = ','
    results = []
    for k,v in countries_dict.items():
        pattern = re.compile(f'{k}')
        check = pattern.findall(row)
        if check:
            results.append(v)
    output = s.join(results)
    return output

Implementación de la funciónn de patterns para todas las columnas del DF. Esto dependerá de la estructura del dataframe. En este ejemplo, se definio aplicar este metodo a **todas las columnas**.

In [ ]:
for col in df.columns:
    df['infotipo'] = df.apply(lambda row: search_patterns_in_columns(row[col],row['infotipo'],pattern_for_it),axis=1)

#### Reemplazo de un ID Viejo por uno Nuevo

El valor del row es convertido a string, y luego compilado y buscado en el valor del row. Si se encuentra, se agrega a una lista de resultados. Finalmente, para evitar duplicaciones, o la incorporacion de valores incorrectos. Se evalua si el Result (lista) tiene un valor mayor a 0, con lo cual, esto implica que existe un valor nuevo a devolver, caso contrario devuelvo el valor viejo denominado "old_id"

*old_id_new_id_dic* Representa una lista con el valor viejo y nuevo mapeado:

old_id_new_id_dic = {

'6001701':'6009800',
'6001702':'6009801',
'6001703':'6009802',
'6001704':'6009803',
'6001705':'6009804',
'6001706':'6009805',
'6001707':'6009806',
'6001708':'6009807',
'6001709':'6009808',
'6001710':'6009809',
'6001711':'6009810',
'6001712':'6009811',
'6001713':'6009812',
'6001714':'6009813',
'6001715':'6009814'

}

*Se implementa con una función anonima para el valor deseado* en este caso se desea Actualizar el valor en userId

**df['TEST_USERID'] =  df.apply(lambda row: update_ids(row['userId']),axis=1)**

**NOTA: el buscar una expresion regular, si el campo tiene otros caracteres, solo sera reemplazado el ID especificado, dejando el resto del string sin actualizar, esto es util al emplearlo para el valor del userid de Time Account**

In [ ]:
def old_id_new_id(row):
    old_id = str(row)
    result = []
    s = ','
    for k,v in old_id_new_id_dic.items():
        pattern = re.compile(f'{k}',re.X)
        check_pattern = pattern.search(old_id)
        if check_pattern:
            new_id = pattern.sub(v,old_id)
            result.append(new_id)
    if len(result) != 0:
        output = s.join(result)
        return output
    else:
        return old_id

## 2) Unificación de archivos

#### Unificación de archivos excels en uno solo, agregando en una columna el nombre de los archivos

In [ ]:
files_xls = []

for f in files:
    if f.endswith(".xlsx"):
        fff = employee_data_path_files + f
        files_xls.append(fff)

df = pd.DataFrame();

for f in files_xls:
    data = pd.read_excel(f, skip_footer=0)
    file_date = os.path.basename(f).replace(".xlsx", "")
    data.index = [file_date] * len(data) #genero el index con el nombre del archivo
    df = df.append(data)

df.to_excel(path_transported + 'unify_employee_data_map_JJ.xlsx', merge_cells = False)

#### Unificación de todas las hojas de un archivo excel en una sola

In [8]:
df_new = pd.concat(pd.read_excel(path2+'EmployeeDataTemplates - 20200703.xlsx', sheet_name=None), ignore_index=False)

In [9]:
df_new.to_excel(path2+'all_data_employeeDataTemplate_test_delete.xlsx') # guardo archivo unificado

## 3)  Control de consistencia entre dos dataframes

Considerando el DF1 como Izquierda y el DF2 como deferecha, se genera un nuevo DF, con los valores que no existen en ambos (distinto de "both"), devolviendo las diferencias ya sea de Izquierda o Derecha segun se reguiera, definiendo este parametro en la propiedad "HOW"

In [ ]:
df1.merge(df2,indicator = True, how='right').loc[lambda x : x['_merge']!='both']

## 4) Modificaciones de dataframes

#### Ejemplo de cuando se quiere generar Rows nuevos a partir de distintos valores separados por "," en una Celda. Ej:

["valor1","Valor2] --> Original


Resultado Final


["valor1]


["valor2"]


In [1]:
df = pd.read_excel(path_transported + 'file.xlsx')

NameError: name 'pd' is not defined

In [444]:
df['campo'] = df['campo'].str.split(',') #hago un split de la celda que tiene el dato separado por ","

In [445]:
new_df = df.apply(pd.Series.explode) #hago un explode de la serie para generar nuevos rows con la columna seleccionada

In [446]:
new_df.to_excel(path_transported + 'file.xlsx') #finalmente guardo el archivo con los nuevos rows

#### Transpose de DT y reseteo de Index

In [ ]:
new_df = df.T.reset_index()

#### Incorporación de columnas vacias con nombres especificos

In [ ]:
# inserto en una posición especifica una columna
df.insert(2,'nuevacolumna1','')
df.insert(3,'nuevacolumna2','')
df.insert(4,'nuevacolumna3','')

#### Generación de dos columnas, a partir del split de una columna

In [ ]:
df[['nuevacolumna1','nuevacolumna2']] = df.Campo.str.split('-',expand=True) 

#### Reemplazo del valor de una columna, a partir de un string

In [ ]:
df['nuevacolumna2'] = df.infotipo.str.replace('PA','')

#### Definición de nuevos nombres de columnas en un DF

In [ ]:
df.set_axis(["columna1", "columna2", "columna3","columna4","columna5","columna6"], axis=1, inplace=False)

#### Agregar valores a un DF a una columna especifica

In [ ]:
for k,v in enumerate(cecos):
        df.at[k+1,'columna_requerida'] = v

#### completar con un valor 0 o 1 a partir del contenido de un string

In [ ]:
# lista de paises, es un array con las listas de paises
lista_paises = ['pais 1', 'pais 2', 'pais 3']

def add_value_to_column_country_not_apply_to(df):
    for p in lista_paises:
        df[p] = np.where(df['columna1'].str.contains(p, na=False), 0,1)

#### Quitar caracteres extraños en todas las columnas

aplicar este proceso a todas las columnas del DataFrame

In [ ]:
for c in columns:
    df[c] = df[c].str.strip()

**Filtros de Datos**

Se define un filtro de datos, y se aplica la busqueda con el metodo "isin"

In [ ]:
workers = ['1','2','3']

In [ ]:
df2 = df[df.worker.isin(workers)]

## 5)  Quitar duplicados en datos

*Función que se utilizo para quitar duplicados de una celda, que tenia los datos en String separados por ","*

In [ ]:
def deduplicate_values(x):
    data = x.split(',') 
    s = ','
    d_duplicate = list(dict.fromkeys(data)) #elimino los duplicados
    data_depured = s.join(d_duplicate) #paso a string la lista
    return data_depured

In [ ]:
#ejemplo de implementación de función
df['columna1'] = df.apply(lambda row: deduplicate_values(row['columna1']),axis=1)

## 6) Control de diferencias entre listas

#### Diferencia entre listados de archivos

files_differences =  set(lista_1_archivos) - set(lista_2_archivos)

In [ ]:
files_differences =  set(dataset_one) - set(dataset_two)
list_difference_control_two = list(files_differences)

#### Archivos en comunes

set(lista_1_archivos).intersection(lista_2_archivos))

Si la función devuelve un **array vacio**, es que no hay archivos en comun

In [ ]:
set(list_of_files_one).intersection(list_of_files_two))

# 7) Lectura de archivo CSV

In [ ]:
#forma 1
df = pd.read_csv(path+'file.csv',sep=r',', engine='python') #caso contratio tira un error

In [ ]:
#forma 2
df = pd.read_csv(root_resultados+'file.csv',delimiter=',',skiprows=1) #salteo la primer fila

# 8) Guardar un archivo CSV

In [ ]:
df.to_csv(path+'new_file.csv',index=False, encoding='utf-8') # considerar el encoding, encado se emplear el archivo en otros sistemas

# 9) Tratamiento de strings

In [10]:
texts = 'FEMX0550COEL27'
num = str(texts[4:8])

In [12]:
num

'0550'

In [14]:
num.replace(f'^[/]','F')

'0550'

**agregar '0' a la izquierda, de un string**

In [1]:
num = '1'

In [2]:
str(num).zfill(8)

'00000001'

## 10) Procesamiento de Archivos Excels

#### Obtener los sheetsname de archivos

In [1]:
xl = pd.ExcelFile('foo.xls')
xl.sheet_names  # see all sheet names
xl.parse(sheet_name)

NameError: name 'pd' is not defined

#### Procesas todos los sheets de un archivo Excel

Cada sheet es una key de un diccionario

dict_keys(['HRP1000', 'HRP1001', 'HRP1002', 'HRP1005', 'HRP1007', 'HRP1008', 'HRP1013', 'HRP9045', 'HRP9047', 'Hoja1'])


In [ ]:
df = pd.read_excel(path+file,sheet_name=None)
df.keys()

In [ ]:
all_dfs['Sheet1'].head()

# 11) Extraccion de Información de un Archivo, a partir de la definición de las columnas a extraer

<span style="color:red">!!! Importante: </span> Estos pasos aplican para un archivo con multiples hojas, sobre lo cual se quiere unificar

En la lista <span style="color:green">**columns_to_colect**</span> defino las columnas que quiero extraer

La función <span style="color:green">**"getColumnNameWithValue**</span> identifica en las columnas del DF si esos valores existen en alguna de las columnas. Esto se da principalemnte en archivos que tienen doble cabecera, o la columna inicia en un row con index distinto de 0.Caso contrario no es necesaria esta operación. Devuelve un array zipeado, para ser utilizado en la siguiente funcion <span style="color:green">**transform_data_colected_into_df**</span>.

La función <span style="color:green">**transform_data_colected_into_df**</span> devuelve el nuevo DataFrame con las columnas identificadas en <span style="color:green">**columns_to_colect**</span>

In [ ]:
columns_to_colect =  ['column1','column2']

#Funcion Auxiliar 
def getColumnNameWithValue ( df , columnValues, sheetname=''):
    columns_to_filters_from_df = []
    rows_index_to_filter = []
    sheetnameWithValue = [] # se agrega para procesar el sheetname que corresponde, si no tiene valor
    for column in df.columns:
        values_to_check = df[column].values
        for val in values_to_check:
            for colValue in columnValues:
                if val == colValue:
                    columns_to_filters_from_df.append(column)
                    ind = df.index[df[column]==val].tolist()
                    rows_index_to_filter.append(ind[0])
                    sheetnameWithValue.append(sheetname)
    index_row_and_column = zip( sheetnameWithValue, rows_index_to_filter, columns_to_filters_from_df)
    return index_row_and_column

#funcion auxiliar
def transform_data_colected_into_df( df, sheet_name, file_name, columnData, column_to_colect):
    
    d = dict.fromkeys(column_to_colect, 0)
    d['sheetname'] = 0
    d['filename'] = 0
    
    data_colected = []
    
    for sheetname, row,col in columnData:
        if sheetname == sheet_name:
            dat = list(df.loc[row+1:,col])
            data_colected.append(dat)
        else:
            pass
    for k,v in zip(d,data_colected):
        d[k] = v
        d['sheetname'] = sheet_name
        d['filename'] = file_name
    #finalmente creo el nuevo dataFrame que ire almacenando en un repositorio para finalmente concatenar todo en un único archivo
    if len(data_colected) != 0:
        new_df = pd.DataFrame(d)
        return new_df
    else:
        return None

dataFrames_to_concat = []

#procesamiento de los archivos
for sheetname in sheetnames :
    print('procesando sheetname {} de archivo {}'.format(sheetname, core_files[2]))
    df = pd.read_excel( root_path + core_files[2] , sheet_name=sheetname)
    colval = getColumnNameWithValue(df,columns_to_colect,sheetname)
    proceded_new_df = transform_data_colected_into_df(df,sheetname,core_files[2],colval,columns_to_colect)
    dataFrames_to_concat.append(proceded_new_df)

final_df = pd.concat(dataFrames_to_concat, ignore_index=True, axis=0)

final_df.to_excel( output +'final_file_with_column_required.xlsx')

## Trabajando con archivos ZIPEADOS

In [ ]:
from zipfile import ZipFile 

In [ ]:
# obtener directorio de un archivo zipeado
with ZipFile(root+files[3], 'r') as zip:
    zip.printdir()

In [ ]:
## leear un archivo csv en un archivo zipeado
df = ''
with ZipFile(root+files[3], 'r') as zip:
    df = pd.read_csv(zip.open('Benefit Enrollment.csv'))